In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
X = np.load("/content/drive/MyDrive/ML/featuresRGB.npy")
Y = np.load("/content/drive/MyDrive/ML/labelRGB.npy")
X = X/255.0

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from sklearn.model_selection import KFold

In [ ]:
def cnn():
    model = Sequential()
    model.add(Convolution2D(64, (3, 3), activation=tf.nn.relu, input_shape=X.shape[1:]))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(64, (3, 3), activation=tf.nn.relu))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(64, (3, 3), activation=tf.nn.relu))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation=tf.nn.relu))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation=tf.nn.sigmoid))

    model.compile(optimizer=tf.keras.optimizers.Adam,
              loss=tf.keras.losses.binary_crossentropy,
              metrics='accuracy' )
    return model

In [ ]:
def mlp():
    model = Sequential()
    model.add(Flatten(input_shape=X.shape[1:])) # input layer
    model.add(Dense(64, activation=tf.nn.relu))
    model.add(Dense(64, activation=tf.nn.relu))
    model.add(Dense(64, activation=tf.nn.relu))
    model.add(Dense(1, activation=tf.nn.sigmoid))
    model.compile(optimizer=tf.keras.optimizers.Adam,
            loss=tf.keras.losses.binary_crossentropy,
            metrics='accuracy' )
    return model

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)

parameter_history = []
parameters = []
for p in parameters:
    #Recording of outputs
    cur_param = [p]
    cur_score = []
    fold_no = 1
    #create model and save weights to be able to reset the model
    model = cnn()
    model.save_weights("/content/drive/MyDrive/ML/weights.h5")

    for train, test in kfold.split(X, Y):
        #reset model
        model.load_weights("/content/drive/MyDrive/ML/weights.h5")

        history = model.fit(X[train], Y[train],
                    batch_size=32,
                    epochs=10,
                    validation_split=0.1,
                    verbose=0)
        
        scores = model.evaluate(X[test], Y[test], verbose=1)

        cur_param.append(history.history)
        cur_score.append(scores)
        fold_no = fold_no + 1
    del model
    cur_param.append(cur_score)
    parameter_history.append(cur_param)

In [ ]:
#data extraction
final = []
for p in parameter_history:
    tl = []
    ta = []
    vl = []
    va = []
    s = []
    for cur in p[1:-1]:
        tl.append(cur['loss'])
        ta.append(cur['accuracy'])
        vl.append(cur['val_loss'])
        va.append(cur['val_accuracy'])
    s = np.mean(p[-1], axis=0)
    tl = np.mean(tl, axis=0)
    ta = np.mean(ta, axis=0)
    vl = np.mean(vl, axis=0)
    va = np.mean(va, axis=0)
    final.append([p[0], tl, ta, vl, va, s])

In [ ]:
np.save("/content/drive/MyDrive/ML/Recordings/olr_cnn_adam.npy", np.array(final,dtype=object))

In [ ]:
final = np.load("/content/drive/MyDrive/ML/Recordings/olr_cnn_adam.npy", allow_pickle=True)

In [ ]:
parameters = []
trainloss = []
trainaccu = []
evalloss = []
evalaccu = []
for p in final:
    parameters.append(str(p[0]))
    trainloss.append(p[1][-1])
    trainaccu.append(p[2][-1])
    evalloss.append(p[-1][0])
    evalaccu.append(p[-1][1])

In [ ]:
#PARAMETER VS LOSS AND ACCURACY GRAPH
parameters = ["ADAM", "SGD", "ADAMAX"]
plt.figure(figsize=(16, 6))
plt.style.use('ggplot')
# Plot loss
plt.subplot(1, 2, 1)
plt.bar(parameters, trainloss )
plt.bar(parameters, evalloss )
#plt.yticks(np.arange(0, 1, 0.1))
plt.ylabel('loss', size=12)
plt.xlabel("Nodes in each layer", size=12)
plt.legend(['train', 'val'], fontsize=14)

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(parameters,trainaccu)
plt.plot(parameters,evalaccu)
#plt.yticks(np.arange(0, 1, 0.1))
plt.ylabel('accuracy', size=12)
plt.xlabel("Nodes in each layer", size=12)
plt.legend(['train', 'val'], fontsize=14)

plt.show()

In [ ]:
#ONLY ONE PARAMETER EPOCH GRAPH
h = final[3]
plt.figure(figsize=(16, 6))
plt.suptitle(f'Parameter {h[0]}')
plt.style.use('ggplot')

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(h[1])
plt.plot(h[3])
plt.xticks(np.arange(1, len(h[1]), 1.0))
plt.ylabel('loss', size=12)
plt.xlabel('epoch', size=12)
plt.legend(['train', 'val'], fontsize=14)

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(h[2][:21])
plt.plot(h[4][:21])
plt.xticks(np.arange(1, 21, 1.0))
plt.ylabel('accuracy', size=12)
plt.xlabel('epoch', size=12)
plt.legend(['train', 'val'], fontsize=14)

plt.show()

In [ ]:
#COMPARE TWO GRAPH
X = final
h = X[0]
k = X[-1]
print(h[0], k[0])
plt.figure(figsize=(16, 6))
plt.suptitle(f'Node Count 16 and 112')
plt.style.use('default')

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(range(1,21),h[1][:20])
plt.plot(range(1,21),k[1][:20])
plt.xticks(np.arange(1, 21, 1.0))
plt.ylabel('loss', size=12)
plt.xlabel('epoch', size=12)
plt.legend(['Node Count 16', 'Node Count 112'], fontsize=14)

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1,21),h[2][:20])
plt.plot(range(1,21),k[2][:20])
plt.xticks(np.arange(1, 21, 1.0))
plt.ylabel('accuracy', size=12)
plt.xlabel('epoch', size=12)
plt.legend(['Node Count 16', 'Node Count 112'], fontsize=14)

plt.show()